In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

2023-04-21 23:54:55.146115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 23:54:55.264139: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [3]:
inputs = Input(shape=(32,32, 3))

# Layer 1
x = Conv2D(32, (3,3), activation='relu', kernel_regularizer=l2(1e-6))(inputs)

# Layer 2
x = Conv2D(32, (3,3), activation='relu', kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 3
x = Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(1e-6))(x)

# Layer 4
x = Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 5
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(1e-6))(x)

# Layer 6
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                             

2023-04-21 23:54:57.471923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-21 23:54:57.474963: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-21 23:54:57.475273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-21 23:54:57.475884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
# 모델 컴파일
opt = RMSprop(learning_rate=1e-4)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip= True
                             )

# 이미지 데이터의 분포를 조정
datagen.fit(x_train)

In [6]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR10", entity="hcim", name='LeNet-relu')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))

callbacks = [wandbCallback]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [7]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)

Epoch 1/200


2023-04-21 23:55:02.296145: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


384/391 [============================>.] - ETA: 0s - loss: 2.1097 - accuracy: 0.2138

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_235459-s7e0224a/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_235459-s7e0224a/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_235459-s7e0224a/files/model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_235459-s7e0224a/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_235459-s7e0224a/files/model-best)... Done. 0.0s


391/391 [==============================] - 11s 28ms/step - loss: 0.6687 - accuracy: 0.7645 - val_loss: 0.7631 - val_accuracy: 0.7365
Epoch 192/200
391/391 [==============================] - 9s 22ms/step - loss: 0.6692 - accuracy: 0.7651 - val_loss: 0.7873 - val_accuracy: 0.7286
Epoch 193/200
391/391 [==============================] - ETA: 0s - loss: 0.6687 - accuracy: 0.7664

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_235459-s7e0224a/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_235459-s7e0224a/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_235459-s7e0224a/files/model-best)... Done. 0.0s


391/391 [==============================] - 10s 25ms/step - loss: 0.6687 - accuracy: 0.7664 - val_loss: 0.7589 - val_accuracy: 0.7402
Epoch 194/200
391/391 [==============================] - 8s 20ms/step - loss: 0.6669 - accuracy: 0.7647 - val_loss: 0.7671 - val_accuracy: 0.7350
Epoch 195/200
391/391 [==============================] - 8s 21ms/step - loss: 0.6661 - accuracy: 0.7661 - val_loss: 0.7698 - val_accuracy: 0.7332
Epoch 196/200
391/391 [==============================] - 8s 20ms/step - loss: 0.6629 - accuracy: 0.7685 - val_loss: 0.7657 - val_accuracy: 0.7355
Epoch 197/200
391/391 [==============================] - 8s 21ms/step - loss: 0.6611 - accuracy: 0.7678 - val_loss: 0.7693 - val_accuracy: 0.7356
Epoch 198/200
391/391 [==============================] - 9s 22ms/step - loss: 0.6599 - accuracy: 0.7696 - val_loss: 0.7682 - val_accuracy: 0.7307
Epoch 199/200
391/391 [==============================] - 8s 21ms/step - loss: 0.6547 - accuracy: 0.7721 - val_loss: 0.7612 - val_accuracy